In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import pickle
import gsdmm
from gsdmm import MovieGroupProcess
from tqdm import tqdm

Ler o CSV criado previamente e converter tokens em listas de tokens

In [12]:
caminho_arquivo = r'C:\Users\Pessoal\Documents\Unifesp\Aries\aries_topic_selector\dados\dadosTwitter\CSV\resitencia_bacteria\tweets_processados.csv'
tweets_df = pd.read_csv(caminho_arquivo)
tweets_df.dropna(axis='columns', inplace=True)
tweets_df['tokens'] = tweets_df.tokens.apply(lambda x:  re.split('\s', x))
docs = tweets_df['tokens'].tolist()

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Pessoal\AppData\Local\Temp\ipykernel_9596\3829233819.py:4: SyntaxWarning: invalid escape sequence '\s'
  tweets_df['tokens'] = tweets_df.tokens.apply(lambda x:  re.split('\s', x))


# Treinamento do Modelo usando o GSDMM

Agora, aplicaremos o GSDMM (Gibbs Sampling Dirichlet Mixture Model) , com os seguintes parâmetros:
- K — É o número máximo de topicos a serem encontrados
- Alpha α - Determina o quão fácil mesas pequenas são removidas
- Beta β - Controla se uma mesa é escolhida por popularidade ou similaridade. Quanto menor, mais popular
- N_iters - Número de vezes em que um Subject é realocado numa nova mesa

In [33]:
mgp = MovieGroupProcess(K=15, alpha=0.08, beta=0.1, n_iters=50)
vocab = set(x for doc in docs for x in doc)
n_terms = len(vocab)
y = mgp.fit(docs, n_terms)


with open("C:\\Users\\Pessoal\\Documents\\Unifesp\Aries\\aries_topic_selector\\modelos_treinados\\10clusters.model", 'wb') as f:
    pickle.dump(mgp, f)
    f.close()

<>:7: SyntaxWarning: invalid escape sequence '\A'
<>:7: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Pessoal\AppData\Local\Temp\ipykernel_9596\993567298.py:7: SyntaxWarning: invalid escape sequence '\A'
  with open("C:\\Users\\Pessoal\\Documents\\Unifesp\Aries\\aries_topic_selector\\modelos_treinados\\10clusters.model", 'wb') as f:


In stage 0: transferred 954 clusters with 15 clusters populated
In stage 1: transferred 578 clusters with 15 clusters populated
In stage 2: transferred 403 clusters with 15 clusters populated
In stage 3: transferred 336 clusters with 15 clusters populated
In stage 4: transferred 255 clusters with 15 clusters populated
In stage 5: transferred 220 clusters with 15 clusters populated
In stage 6: transferred 228 clusters with 15 clusters populated
In stage 7: transferred 185 clusters with 15 clusters populated
In stage 8: transferred 177 clusters with 15 clusters populated
In stage 9: transferred 186 clusters with 15 clusters populated
In stage 10: transferred 187 clusters with 15 clusters populated
In stage 11: transferred 185 clusters with 15 clusters populated
In stage 12: transferred 171 clusters with 15 clusters populated
In stage 13: transferred 178 clusters with 15 clusters populated
In stage 14: transferred 164 clusters with 15 clusters populated
In stage 15: transferred 162 cluste

# Explorando os Tópicos

Definiremos algumas funções para ajudar a explorar os tópicos

In [9]:
def top_words(cluster_word_distribution, top_cluster, values):
    '''prints the top words in each cluster'''
    for cluster in top_cluster:
        sort_dicts =sorted(mgp.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print('Cluster %s : %s'%(cluster,sort_dicts))
        print(' — — — — — — — — —')

def cluster_importance(mgp):
    '''returns a word-topic matrix[phi] where each value represents
    the word importance for that particular cluster;
    phi[i][w] would be the importance of word w in topic i.
    '''
    n_z_w = mgp.cluster_word_distribution
    beta, V, K = mgp.beta, mgp.vocab_size, mgp.K
    phi = [{} for i in range(K)]
    for z in range(K):
        for w in n_z_w[z]:
            phi[z][w] = (n_z_w[z][w]+beta)/(sum(n_z_w[z].values())+V*beta)
    return phi

def topic_allocation(df, docs, mgp, topic_dict):
    '''allocates all topics to each document in original dataframe,
    adding two columns for cluster number and cluster description'''
    topic_allocations = []
    for doc in tqdm(docs):
        topic_label, score = mgp.choose_best_label(doc)
        topic_allocations.append(topic_label)

    df['cluster'] = topic_allocations

    df['topic_name'] = df.cluster.apply(lambda x: get_topic_name(x, topic_dict))
    print('Complete. Number of documents with topic allocated: {}'.format(len(df)))

def get_topic_name(doc, topic_dict):
    '''returns the topic name string value from a dictionary of topics'''
    topic_desc = topic_dict[doc]
    return topic_desc

Insight estatístico

In [34]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)
print('*'*20)
# topics sorted by the number of documents they are allocated to
top_index = doc_count.argsort()[-10:][::-1]
print('Most important clusters (by number of docs inside):',top_index)
print('*'*20)
# show the top 5 words in term frequency for each cluster 
topic_indices = np.arange(start=0, stop=len(doc_count), step=1)
top_words(mgp.cluster_word_distribution, topic_indices, 5)

Number of documents per topic : [ 48  53  54  32  23  67  30  33  29  19  94 360  45  36 316]
********************
Most important clusters (by number of docs inside): [11 14 10  5  2  1  0 12 13  7]
********************
Cluster 0 : [('criar', 15), ('bolsonaro', 8), ('ser', 7), ('forte', 7), ('brasil', 7)]
 — — — — — — — — —
Cluster 1 : [('infecção', 13), ('tratamento', 12), ('desenvolver', 10), ('tratar', 8), ('ério', 8)]
 — — — — — — — — —
Cluster 2 : [('mecanismo', 12), ('ser', 11), ('gene', 10), ('parede', 9), ('celular', 9)]
 — — — — — — — — —
Cluster 3 : [('una', 20), ('resistència', 14), ('per', 13), ('als', 9), ('antibiòtics', 7)]
 — — — — — — — — —
Cluster 4 : [('gene', 8), ('coisa', 6), ('humano', 4), ('lagarta', 3), ('soja', 3)]
 — — — — — — — — —
Cluster 5 : [('gene', 12), ('reduzir', 10), ('estudo', 9), ('base', 8), ('infecção', 8)]
 — — — — — — — — —
Cluster 6 : [('anel', 5), ('outro', 5), ('de', 5), ('ocorrer', 4), ('tratar', 4)]
 — — — — — — — — —
Cluster 7 : [('alto', 1

Se quisermos ver a importância de algumas palavras , usando a função cluster_importance

Agora, decidimos o que é  cada tópico

# metricas

In [35]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
import numpy as np

# Supondo que:
# - mgp é o modelo treinado com fit()
# - docs é uma lista de listas de tokens (tokenized_docs)
docs = tweets_df["tokens"].tolist()
# Obter tópicos (palavras mais frequentes por cluster)
def extract_top_words(cluster_word_dist, top_n=10):
    topic_words = []
    for cluster_id in range(len(cluster_word_dist)):
        sorted_words = sorted(cluster_word_dist[cluster_id].items(), key=lambda kv: kv[1], reverse=True)
        words = [word for word, _ in sorted_words[:top_n]]
        topic_words.append(words)
    return topic_words

# Extrair palavras por tópico
top_n = 10
topic_words = extract_top_words(mgp.cluster_word_distribution, top_n=top_n)

# Remover tópicos vazios (podem acontecer se o cluster não tiver documentos)
topic_words = [topic for topic in topic_words if len(topic) > 0]

# Criar dicionário e corpus no formato Gensim
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(text) for text in docs]

# Calcular Coerência
coherence_model = CoherenceModel(
    topics=topic_words,
    texts=docs,
    dictionary=dictionary,
    coherence='c_v'
)
coherence_score = coherence_model.get_coherence()

# Calcular Diversidade
all_words = [word for topic in topic_words for word in topic]
unique_words = set(all_words)
diversity_score = len(unique_words) / len(all_words)

# Resultados
print(f"[GSDMM] Coerência (C_V): {coherence_score:.4f}")
print(f"[GSDMM] Diversidade: {diversity_score:.4f}")


[GSDMM] Coerência (C_V): 0.3471
[GSDMM] Diversidade: 0.7467


In [14]:
topic_dict = {}
topic_names = ['Cluster 0,',
                'Cluster 1,',
                'Cluster 2,',
                'Cluster 3,',
                'Cluster 4,',
                'Cluster 5,',
                'Cluster 6,',
                'Cluster 7,',
                'Cluster 8,',
                'Cluster 9,']

for i, topic_num in enumerate(topic_indices):
 topic_dict[topic_num]=topic_names[i]



topic_allocation(tweets_df, docs, mgp, topic_dict)

100%|██████████| 19571/19571 [00:06<00:00, 3124.01it/s]


Complete. Number of documents with topic allocated: 19571


In [15]:
tweets_df.to_excel('sttm_10topics_results.xlsx', index = False,  header=True)

# Aplicação do Método para Notícias do G1

In [7]:
caminho_arquivo = r'C:\Users\Pessoal\Documents\Unifesp\Aries\aries_topic_selector\Noise_remover\noticias_processados.csv'
news_df = pd.read_csv(caminho_arquivo)
news_df.dropna(axis='columns', inplace=True)
news_df['tokens'] = news_df.tokens.apply(lambda x:  re.split('\s', x))
docs = news_df['tokens'].tolist()

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Pessoal\AppData\Local\Temp\ipykernel_12864\3873121176.py:4: SyntaxWarning: invalid escape sequence '\s'
  news_df['tokens'] = news_df.tokens.apply(lambda x:  re.split('\s', x))


# Treinamento do Modelo usando o GSDMM

In [8]:
mgp2 = MovieGroupProcess(K=10, alpha=0.1, beta=0.1, n_iters=30)
vocab = set(x for doc in docs for x in doc)
n_terms = len(vocab)
y = mgp2.fit(docs, n_terms)


with open("C:\\Users\\Pessoal\\Documents\\Unifesp\Aries\\aries_topic_selector\\modelos_treinados\\10clusters.model", 'wb') as f:
    pickle.dump(mgp2, f)
    f.close()

<>:7: SyntaxWarning: invalid escape sequence '\A'
<>:7: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Pessoal\AppData\Local\Temp\ipykernel_12864\2481954221.py:7: SyntaxWarning: invalid escape sequence '\A'
  with open("C:\\Users\\Pessoal\\Documents\\Unifesp\Aries\\aries_topic_selector\\modelos_treinados\\10clusters.model", 'wb') as f:


In stage 0: transferred 168 clusters with 9 clusters populated
In stage 1: transferred 23 clusters with 6 clusters populated
In stage 2: transferred 6 clusters with 6 clusters populated
In stage 3: transferred 1 clusters with 6 clusters populated
In stage 4: transferred 0 clusters with 6 clusters populated
In stage 5: transferred 2 clusters with 6 clusters populated
In stage 6: transferred 1 clusters with 6 clusters populated
In stage 7: transferred 0 clusters with 6 clusters populated
In stage 8: transferred 1 clusters with 6 clusters populated
In stage 9: transferred 0 clusters with 6 clusters populated
In stage 10: transferred 1 clusters with 6 clusters populated
In stage 11: transferred 1 clusters with 6 clusters populated
In stage 12: transferred 0 clusters with 6 clusters populated
In stage 13: transferred 2 clusters with 6 clusters populated
In stage 14: transferred 1 clusters with 6 clusters populated
In stage 15: transferred 1 clusters with 6 clusters populated
In stage 16: tr

Insight estatístico

In [36]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)
print('*'*20)
# topics sorted by the number of documents they are allocated to
top_index = doc_count.argsort()[-10:][::-1]
print('Most important clusters (by number of docs inside):',top_index)
print('*'*20)
# show the top 5 words in term frequency for each cluster 
topic_indices = np.arange(start=0, stop=len(doc_count), step=1)
top_words(mgp.cluster_word_distribution, topic_indices, 5)

Number of documents per topic : [ 48  53  54  32  23  67  30  33  29  19  94 360  45  36 316]
********************
Most important clusters (by number of docs inside): [11 14 10  5  2  1  0 12 13  7]
********************
Cluster 0 : [('criar', 15), ('bolsonaro', 8), ('ser', 7), ('forte', 7), ('brasil', 7)]
 — — — — — — — — —
Cluster 1 : [('infecção', 13), ('tratamento', 12), ('desenvolver', 10), ('tratar', 8), ('ério', 8)]
 — — — — — — — — —
Cluster 2 : [('mecanismo', 12), ('ser', 11), ('gene', 10), ('parede', 9), ('celular', 9)]
 — — — — — — — — —
Cluster 3 : [('una', 20), ('resistència', 14), ('per', 13), ('als', 9), ('antibiòtics', 7)]
 — — — — — — — — —
Cluster 4 : [('gene', 8), ('coisa', 6), ('humano', 4), ('lagarta', 3), ('soja', 3)]
 — — — — — — — — —
Cluster 5 : [('gene', 12), ('reduzir', 10), ('estudo', 9), ('base', 8), ('infecção', 8)]
 — — — — — — — — —
Cluster 6 : [('anel', 5), ('outro', 5), ('de', 5), ('ocorrer', 4), ('tratar', 4)]
 — — — — — — — — —
Cluster 7 : [('alto', 1

In [ ]:
topic_dict = {}
topic_names = ['Cluster 0,',
                'Cluster 1,',
                'Cluster 2,',
                'Cluster 3,',
                'Cluster 4,',
                'Cluster 5,',
                'Cluster 6,',
                'Cluster 7,',
                'Cluster 8,',
                'Cluster 9,']

for i, topic_num in enumerate(topic_indices):
 topic_dict[topic_num]=topic_names[i]



topic_allocation(tweets_df, docs, mgp, topic_dict)

100%|██████████| 4268/4268 [00:01<00:00, 3300.27it/s]

Complete. Number of documents with topic allocated: 4268


In [ ]:
tweets_df.to_excel('sttm_10topics_results.xlsx', index = False,  header=True)